First we record a user input

In [5]:
import pyaudio
import wave
import time

# Audio settings
sample_format = pyaudio.paInt16
channels = 1  # Mono audio input
sample_rate = 44100  # Standard sample rate (Hz)
chunk_size = 1024  # Size of each audio chunk
record_duration = 15  # Duration to record in seconds

# Initialize PyAudio
p = pyaudio.PyAudio()

# Open an audio stream for input
stream = p.open(format=sample_format,
               channels=channels,
               rate=sample_rate,
               input=True,
               frames_per_buffer=chunk_size)

# Create an empty list to store the audio frames
frames = []

print("Recording...")

# Record audio for the specified duration
start_time = time.time()
while (time.time() - start_time) < record_duration:
    data = stream.read(chunk_size)
    frames.append(data)

# Stop recording
print("Finished recording.")
stream.stop_stream()
stream.close()
p.terminate()

# Save the recorded audio data to a WAV file
with wave.open("input.wav", "wb") as wf:
    wf.setnchannels(channels)
    wf.setsampwidth(p.get_sample_size(sample_format))
    wf.setframerate(sample_rate)
    wf.writeframes(b"".join(frames))

Recording...
Finished recording.


Using Spotify's Basic-Pitch for pitch estimation

In [1]:
#convert wav to midi 
from basic_pitch.inference import predict
from basic_pitch import ICASSP_2022_MODEL_PATH

model_output, midi_data, note_events = predict("song.wav", ICASSP_2022_MODEL_PATH)
print(note_events)


Predicting MIDI for song.wav...
[(422.64316190476194, 422.79409160997733, 41, 0.5582025, [1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), (422.2716426303855, 422.56189206349205, 50, 0.5810666, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), (421.9349532879819, 422.15554285714285, 48, 0.5532297, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), (421.69114376417235, 422.2019827664399, 57, 0.38202155, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), (421.69114376417235, 422.027833106576, 53, 0.4435131, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), (421.4241142857143, 421.78402358276645, 69, 0.47636816, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), (421.4241142857143, 421.7956335600907, 41, 0.38686198, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1,

Saving the file

In [3]:
#save the mid file
def save_midi(midi_data, output_path):
    with open(output_path, 'wb') as output_file:
        midi_data.write(output_file)

save_midi(midi_data, "input.mid")

import pretty_midi
pm = pretty_midi.PrettyMIDI('input.mid')
print(pm.instruments[0].notes)

[Note(start=0.115909, end=0.254545, pitch=58, velocity=75), Note(start=0.254545, end=0.593182, pitch=60, velocity=104), Note(start=0.593182, end=0.800000, pitch=60, velocity=96), Note(start=0.929545, end=1.090909, pitch=55, velocity=73), Note(start=0.904545, end=1.138636, pitch=57, velocity=41), Note(start=1.022727, end=1.172727, pitch=65, velocity=53), Note(start=1.138636, end=1.277273, pitch=57, velocity=68), Note(start=0.904545, end=1.277273, pitch=60, velocity=48), Note(start=1.277273, end=1.438636, pitch=57, velocity=91), Note(start=1.311364, end=1.706818, pitch=41, velocity=66), Note(start=1.438636, end=1.718182, pitch=57, velocity=88), Note(start=1.729545, end=2.184091, pitch=60, velocity=64), Note(start=2.113636, end=2.450000, pitch=41, velocity=59), Note(start=2.370455, end=2.520455, pitch=65, velocity=42), Note(start=1.718182, end=2.531818, pitch=57, velocity=69), Note(start=2.531818, end=2.718182, pitch=65, velocity=41), Note(start=2.602273, end=2.856818, pitch=41, velocity=

{Optional} Stretch(pad) the mid file

In [3]:
import pretty_midi

def stretch_midi(input_midi_file, output_midi_file, stretch_factor):
    # Load the MIDI file
    midi_data = pretty_midi.PrettyMIDI(input_midi_file)

    # Stretch the timing of notes in each instrument
    for instrument in midi_data.instruments:
        for note in instrument.notes:
            note.start *= stretch_factor
            note.end *= stretch_factor

    # Save the stretched MIDI file
    midi_data.write(output_midi_file)

    

In [22]:
from fastdtw import fastdtw

def get_dtw_distance(input_midi_data, output_midi_data):
    input_pitches=[]
    for note in input_midi_data.instruments[0].notes:
        input_pitches.append(note.pitch)
    segment_pitches=[]
    for note in output_midi_data:
        segment_pitches.append(note.pitch)
    distance, path = fastdtw(input_pitches, segment_pitches)
    
    return distance


In [19]:
import os
import pretty_midi

#now we have to compare the recorded midi file with the original midi file
input_midi_data = pretty_midi.PrettyMIDI("input.mid")

#for each midi file in the dataset
for filename in os.listdir("test_dataset"):
    print("Comparing input with " + filename + "...")
    original_midi_data = pretty_midi.PrettyMIDI("test_dataset/"+filename)

    #cut the original midi file into segments of 15 seconds each
    segment_duration = 15
    best_segment = None
    smallest = 1000000
    for i in range(0, len(original_midi_data.instruments[0].notes)-segment_duration,segment_duration):
        segment = original_midi_data.instruments[0].notes[i:i+segment_duration]
        dist = get_dtw_distance(input_midi_data, segment)
        if dist < smallest:
            smallest = dist
            best_segment = segment
    print("Distance: " + str(smallest) + " Best segment: " + str(best_segment))


Comparing input with All_Out_Of_Love.1.mid...
Distance: 195.0 Best segment: [Note(start=134.036523, end=134.053725, pitch=35, velocity=59), Note(start=134.311752, end=134.328954, pitch=51, velocity=53), Note(start=134.311752, end=134.328954, pitch=35, velocity=112), Note(start=134.862210, end=134.879412, pitch=51, velocity=22), Note(start=135.412668, end=135.429870, pitch=51, velocity=82), Note(start=135.963126, end=135.980328, pitch=51, velocity=43), Note(start=136.513584, end=136.530786, pitch=51, velocity=82), Note(start=136.513584, end=136.530786, pitch=35, velocity=124), Note(start=137.614500, end=137.631702, pitch=51, velocity=82), Note(start=138.715416, end=138.732618, pitch=51, velocity=82), Note(start=139.816332, end=139.833534, pitch=51, velocity=98), Note(start=140.917248, end=140.934450, pitch=51, velocity=36), Note(start=142.018164, end=142.035366, pitch=49, velocity=82), Note(start=142.018164, end=142.035366, pitch=35, velocity=124), Note(start=142.568622, end=142.585824,